# Introduction to Memory Profiling

> Objectives:
> * Be introduced to memory profiling using different tools
> * Some small introduction to time profiling in IPython too


## ipython_memwatcher

Our recommended way to profile memory consumption for this tutorial will be [ipython_memwatcher](https://pypi.python.org/pypi/ipython_memwatcher):


In [1]:
from ipython_memwatcher import MemWatcher
mw = MemWatcher()
mw.start_watching_memory()

In [1] used 0.129 MiB RAM in 0.003s, peaked 0.000 MiB above current, total RAM usage 28.285 MiB


In [2]:
# Let's create a big object
a = [i for i in range(1000*1000)]

In [2] used 32.492 MiB RAM in 0.151s, peaked 0.000 MiB above current, total RAM usage 60.777 MiB


In [3]:
# Get some measurements from the last executed cell:
meas = mw.measurements
meas

Measurements(memory_delta=32.4921875, time_delta=0.15124106407165527, memory_peak=0, memory_usage=60.77734375)

In [3] used 0.051 MiB RAM in 0.018s, peaked 0.000 MiB above current, total RAM usage 60.828 MiB


In [4]:
# MemWatcher.measurements is a named tuple.  We can easily get info out of it:
meas.memory_delta

32.4921875

In [4] used 0.016 MiB RAM in 0.003s, peaked 0.000 MiB above current, total RAM usage 60.844 MiB


In [5]:
# This takes betweed 32 ~ 35 bytes per element:
meas.memory_delta * (2**20) / 1e6

34.070528

In [5] used 0.004 MiB RAM in 0.003s, peaked 0.000 MiB above current, total RAM usage 60.848 MiB


In [6]:
# What are these elements made from?
type(a[0])

int

In [6] used 0.008 MiB RAM in 0.003s, peaked 0.000 MiB above current, total RAM usage 60.855 MiB


In [7]:
# How much memory take an int?
# Beware: the size below will depend on whether you are using a 32-bit or 64-bit Python
import sys
sys.getsizeof(a[0])

24

In [7] used 0.000 MiB RAM in 0.003s, peaked 0.000 MiB above current, total RAM usage 60.855 MiB


Ok.  On 64-bits, that means that the int object allocates 8 bytes for the integer value, and 16 bytes for other metadata (Python object).  But 24 is quite less than 32~35.  Where this overhead comes from?

## objgraph

In [9]:
# Let's introduce the objgraph package and see
b = [1,2,3]
import objgraph
objgraph.show_refs([b], filename='simple-list.png')
#objgraph.show_refs([b])
from IPython.core.display import Image 
Image('simple-list.png')

Graph written to /var/folders/2q/970l17jj7sl3mb_jgmmrnw2r0000gn/T/objgraph-ggf1r2aq.dot (4 nodes)
Graph viewer (xdot) and image renderer (dot) not found, not doing anything else


In [9] used -3.867 MiB RAM in 0.188s, peaked 0.000 MiB above current, total RAM usage 63.617 MiB


So, the list is an structure that takes a pointer (8 bytes in 64-bit platforms) per every element in the list.  If we add this to the 24 bytes per int, then we have 32 bytes per element, which is close to the computed 32~35 bytes above.  The remaining difference is probably due to how Python handles memory internally (over-allocation). 

## memory_profiler

[memory_profiler](https://pypi.python.org/pypi/memory_profiler) is a basic module for memory profiling that many others use (like the `ipython_memwatcher` above) and it interacts well with ipython, so it is worth to see how it works:

In [10]:
%load_ext memory_profiler

In [10] used 0.023 MiB RAM in 0.002s, peaked 0.000 MiB above current, total RAM usage 63.641 MiB


In [11]:
# Use %memit magic command exposed by memory_profiler
%memit b = [i for i in range(1000*1000)]

peak memory: 128.53 MiB, increment: 64.89 MiB
In [11] used 37.098 MiB RAM in 0.547s, peaked 5.984 MiB above current, total RAM usage 100.738 MiB


Please note that the `peak_memory` in this case is different than the `peaked_memory` reported by ipython_memwatcher package.

## Guppy

Guppy is nice for having an overview of how different structures are using our memory:

In [12]:
from guppy import hpy; hp=hpy()
hp.heap()

ImportError: No module named 'guppy'

In [12] used 6.793 MiB RAM in 0.209s, peaked 0.000 MiB above current, total RAM usage 107.531 MiB


In [20]:
del b
hp.heap()

NameError: name 'hp' is not defined

In [20] used -31.539 MiB RAM in 0.053s, peaked 31.539 MiB above current, total RAM usage 70.094 MiB


In [13]:
# Size of the list (beware, this does not include the contents!)
hp.iso(a)

NameError: name 'hp' is not defined

In [13] used -4.910 MiB RAM in 0.006s, peaked 0.000 MiB above current, total RAM usage 102.621 MiB


## %time and %timeit

In [21]:
# IPython provides a magic command to see how much time a command takes to run
%time asum = sum(a)

CPU times: user 23.3 ms, sys: 8.1 ms, total: 31.4 ms
Wall time: 34.7 ms
In [21] used 0.020 MiB RAM in 0.038s, peaked 0.000 MiB above current, total RAM usage 70.113 MiB


Note that `%time` offers quite detailed statistics on the time spent.

Also, the time reported by MemoryWatcher has a typical overhead of 3~10 ms over the time reported by %time, so when the times to measure are about this order then it is better to rely on the %time (or %timeit below) values.  

In [ ]:
# We have another way to measure timings doing several loops and getting the mean
%timeit bsum = sum(a)

In [ ]:
# However, one must notice that %timeit does not return the result of expressions
bsum

Interestingly, %timeit allows to retrieve the measured times in loops with the -o flag:

In [ ]:
t = %timeit -o sum(a)
print(t.all_runs)
print(t.best)

And one can specify the number of loops (-n) and the number of repetitions (-r):

In [ ]:
t = %timeit -r1 -n1 -o sum(a)
print(t.all_runs)
print(t.best)

### Exercise 1

Provided a dictionary like:

```
d = dict(("key: %i"%i, i*2) for i in a)
```

Try to guess how much RAM it uses using the techniques introduced above.

Why do you think it takes more space than a list?

*Hint*: Use the `objgraph` package on a short dictionary so as to better see the data structure.  In case you cannot get `objgraph` to work, every entry in a dictionary has pointers to two objects: key and value. 

## Solution

In [22]:
d = dict(("key: %i"%i, i*2) for i in a)

In [22] used 129.254 MiB RAM in 1.538s, peaked 0.000 MiB above current, total RAM usage 199.367 MiB


In [23]:
objgraph.show_refs([d], filename='dict.png')
from IPython.core.display import Image 
Image('dict.png')

Graph written to /var/folders/2q/970l17jj7sl3mb_jgmmrnw2r0000gn/T/objgraph-nimf8yc7.dot (11 nodes)
Image renderer (dot) not found, not doing anything else


In [23] used 5.383 MiB RAM in 0.830s, peaked 7.633 MiB above current, total RAM usage 204.750 MiB


In [24]:
# Compute the size of key + value
sys.getsizeof("key: 100000") + sys.getsizeof(1)

88

In [24] used -6.223 MiB RAM in 0.003s, peaked 0.000 MiB above current, total RAM usage 198.527 MiB


In [25]:
# Compute the size of a key + value + pointers for all elements in MB
(sys.getsizeof("key: 100000") + sys.getsizeof(1) + 16) * 1000*1000 / 2**20.

99.18212890625

In [25] used -6.086 MiB RAM in 0.004s, peaked 0.000 MiB above current, total RAM usage 192.441 MiB


In [26]:
# Using guppy
hp.iso(d)

NameError: name 'hp' is not defined

In [26] used -1.988 MiB RAM in 0.008s, peaked 0.000 MiB above current, total RAM usage 190.453 MiB


So, guppy is telling us that just the dictionary structure is taking ~50 MB, whereas the contents alone are taking ~70MB, so we should have expected the dictionary to consume ~120 MB.  However, our `MemWatcher` instance is reporting just 94 MB.  Take away lesson: measuring memory consumption in Python is tricky but using proper tools we can still have valuable hints!